In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import os
import shutil

In [4]:

# Define paths
original_dataset_dir = r"animal\raw-img"
base_dir = r"C:\Users\Leapfrog\Desktop\Others\Mtech AI\DNN\Assignment 2"

# Create directories for training and validation splits
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)

# Iterate through each category (class) in the raw-img directory
for category in os.listdir(original_dataset_dir):
    category_dir = os.path.join(original_dataset_dir, category)
    
    # Ensure category_dir is a directory
    if os.path.isdir(category_dir):
        images = [f for f in os.listdir(category_dir) if os.path.isfile(os.path.join(category_dir, f))]
        
        # Debugging: Print out the number of images found
        print(f"Processing category: {category} - {len(images)} images found")

        # Skip this category if there are no images
        if len(images) == 0:
            print(f"No images found in {category}, skipping this category.")
            continue
        
        # Split the images into training and validation sets
        train_images, validation_images = train_test_split(images, test_size=0.2, random_state=42)
        
        # Create category directories in train and validation folders
        train_category_dir = os.path.join(train_dir, category)
        os.makedirs(train_category_dir, exist_ok=True)
        validation_category_dir = os.path.join(validation_dir, category)
        os.makedirs(validation_category_dir, exist_ok=True)
        
        # Copy the images
        for image in train_images:
            src = os.path.join(category_dir, image)
            dst = os.path.join(train_category_dir, image)
            shutil.copy2(src, dst)
        
        for image in validation_images:
            src = os.path.join(category_dir, image)
            dst = os.path.join(validation_category_dir, image)
            shutil.copy2(src, dst)

print("Data successfully split into training and validation sets!")

Processing category: cane - 4863 images found
Processing category: cavallo - 2623 images found
Processing category: elefante - 1446 images found
Processing category: farfalla - 2112 images found
Processing category: gallina - 3098 images found
Processing category: gatto - 1668 images found
Processing category: mucca - 1866 images found
Processing category: pecora - 1820 images found
Processing category: ragno - 4821 images found
Processing category: scoiattolo - 1862 images found
Data successfully split into training and validation sets!


In [6]:
# Define parameters
image_size = (224, 224)
batch_size = 32

# Image data generator for loading images
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

# Load dataset from directory
train_generator = datagen.flow_from_directory(
    r"train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    r"validation",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)


Found 16755 images belonging to 10 classes.
Found 1044 images belonging to 10 classes.


In [ ]:
# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of VGG16
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 22s 0us/step
Epoch 1/5


C:\Users\Leapfrog\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


524/524 ━━━━━━━━━━━━━━━━━━━━ 1221s 2s/step - accuracy: 0.5478 - loss: 2.4052 - val_accuracy: 0.7902 - val_loss: 0.6525
Epoch 2/5
524/524 ━━━━━━━━━━━━━━━━━━━━ 1290s 2s/step - accuracy: 0.7486 - loss: 0.7351 - val_accuracy: 0.8324 - val_loss: 0.5297
Epoch 3/5
524/524 ━━━━━━━━━━━━━━━━━━━━ 1368s 3s/step - accuracy: 0.8027 - loss: 0.5847 - val_accuracy: 0.8314 - val_loss: 0.5392
Epoch 4/5
404/524 ━━━━━━━━━━━━━━━━━━━━ 5:17 3s/step - accuracy: 0.7997 - loss: 0.5683

In [ ]:
import matplotlib.pyplot as plt

history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

# Plotting accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Plotting loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.show()


In [12]:
# Extract features from the training data using the trained model
feature_extractor = Model(inputs=model.input, outputs=model.layers[-3].output)  # Output from the dense layer before softmax

train_features = feature_extractor.predict(train_generator)
train_labels = train_generator.classes

# Flatten the features for SVM
train_features = train_features.reshape(train_features.shape[0], -1)

# Extract features from the validation data
validation_features = feature_extractor.predict(validation_generator)
validation_labels = validation_generator.classes

# Flatten the features for SVM
validation_features = validation_features.reshape(validation_features.shape[0], -1)


524/524 ━━━━━━━━━━━━━━━━━━━━ 3045s 6s/step
33/33 ━━━━━━━━━━━━━━━━━━━━ 161s 5s/step


In [ ]:
# Initialize the SVM classifier
svm = SVC(kernel='linear')

# Train the SVM model
svm.fit(train_features, train_labels)


In [ ]:
# Predict using the SVM classifier
y_pred = svm.predict(validation_features)

# Evaluate the performance
accuracy = accuracy_score(validation_labels, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print a classification report
print(classification_report(validation_labels, y_pred, target_names=validation_generator.class_indices.keys()))
